<a href="https://colab.research.google.com/github/TanViCh0UhAn12/ML/blob/main/ANN_A1Classi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np 
import pandas as pd

from sklearn import metrics
from sklearn import preprocessing

data = pd.read_csv("/content/Churn_Modelling.csv")


In [11]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
data.drop(columns=['RowNumber','CustomerId', 'Surname', 'CreditScore', 'Gender','Geography','Age'], inplace=True)

In [13]:
data['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [15]:
X = data.drop('Exited', axis=1)
y = data['Exited']

X.head(4), y.head(4)

(   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
 0       2       0.00              1          1               1   
 1       1   83807.86              1          0               1   
 2       8  159660.80              3          1               0   
 3       1       0.00              2          0               0   
 
    EstimatedSalary  
 0        101348.88  
 1        112542.58  
 2        113931.57  
 3         93826.63  ,
 0    1
 1    0
 2    1
 3    0
 Name: Exited, dtype: int64)

In [18]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

col_trans = ColumnTransformer([
  
    ('ss', StandardScaler(), ['Tenure','Balance','EstimatedSalary'])], remainder='passthrough')


In [19]:
X = col_trans.fit_transform(X)

In [20]:
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

smote_enn = SMOTEENN(random_state=42)
X, y = smote_enn.fit_resample(X, y)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [23]:
!pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, n_jobs=-1)

search_space = {
        "max_depth": Integer(10, 25), 
        "n_estimators": Integer(10, 50)
    }

search = BayesSearchCV(rf, search_space, n_iter=50, scoring="accuracy", cv=5, verbose= 0)
search.fit(X_train, y_train)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 9.7 MB/s eta 0:00:00


/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.9/dist-packa

BayesSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
              scoring='accuracy',
              search_spaces={'max_depth': Integer(low=10, high=25, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=10, high=50, prior='uniform', transform='normalize')})

In [25]:
rf.set_params(**search.best_params_)
rf.fit(X_train, y_train)

print(rf.score(X_train, y_train))  # performance on train data)
print(rf.score(X_test, y_test))  # performance on test data)

1.0
0.9498945147679325


In [26]:
y_pred_train = rf.predict(X_train)

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

accuracy = metrics.accuracy_score(y_train, y_pred_train)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_train,y_pred_train)
print('Confusion Matrix: \n', cm)
print(classification_report(y_train, y_pred_train))

Accuracy: 1.00
Confusion Matrix: 
 [[3614    0]
 [   0 3969]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3614
           1       1.00      1.00      1.00      3969

    accuracy                           1.00      7583
   macro avg       1.00      1.00      1.00      7583
weighted avg       1.00      1.00      1.00      7583

